I was working through a diffusion problem and thought that **Python** and a package for dealing with units and unit conversions called **pint** would be usefull.

I'm using the **Anaconda** distribution **Python**. To use **pint**, I needed to install it using the **Anaconda Prompt**.

```
> pip install pint
```

The problem I am working on solving involes the diffusion of nitrogen gas (N<sub>2</sub>) into a thin plate of iron. 

When &alpha;-iron put in a nitrogen atmosphere, the conventration of nitrogen in the iron, _C<sub>N</sub>_ is a function of the nitrogen pressure $P_{N_2}$ accoding to the relationship:

$$C_{N} = 4.9 \times 10^{-6} \sqrt{P_{N_2}} exp{\frac{Qn}{RT}} $$

Where:

$Q_n = 37,600 \frac{J}{mol}$

$R=8.31 \frac{J}{mol-K}$

$T$ is the temperature in Kelvin.

At 300 &deg;C the nitrogen gas pressure on one side of an iron plate is 0.10 MPa. On the other side of the iron plate, the nitrogen gas pressure is 5.0 MPa. The iron plate is a 1.5 mm thick.  Assume the pre-exponential term _D<sub>0</sub>_ and the activation energy of diffusion of nitrogen in carbon, _Q<sub>d</sub>_ are equal to the values below:

$D_0 = 5 \times 10^{-7} \frac{m^2}{s}$

$Q_d = 77,000 \frac{J}{mol} $

Calculate the diffusion flux, _J_ through the plate using Fick's First Law of Diffusion:

$$ J = -D \frac{dC}{dx} $$

We have a couple different quantities and a couple of different units to handle to solve this problem. I started out importing **pint** and creating a ```UnitRegistry``` object. I'm also going to need the ```exp``` (e raised to a power) and ```sqrt``` (square root) functions from the math module.

In [1]:
import pint
from math import exp, sqrt
u = pint.UnitRegistry()

Let's start with the temperature, T = 300 &deg;C.

Temperature units in &deg;F and &deg;C are relative units with an off-set scale. They are not _multiplicative units_.  _Non-multiplicatve units_ are handled by **pint** a little differently compared to regular _multiplicative_ units.  

To create a variable with  a degree C units we instantiate a ```Quantity``` object and pass in the temperature in &deg;C along with the unit (```u.degC```). We can convert the temperature to Kelvin(K) using the ```.ito``` method. 

Since we want to do some mulipication, division and exponentiation with our temperature, we need to convert the temperature to _multiplicative_ unit. **Pint** has two versions of the temperature unit in Kelvin. There is the _non-multiplicative_ type ```degK``` and the _multiplicative_ type ```kelvin```. 

We convert the temperature to the _multiplicative_ type ```kelvin``` by pulling out the magnitude (the number part without the ```degK``` unit) from the ```T``` variable and multiplying it by the ```kevlin``` unit from **pint**.

In [2]:
Q_ = u.Quantity
T = Q_(300, u.degC)
print(T)
T.ito('degK')
print('T = {}'.format(T))
T = T.magnitude * u.kelvin
print(T)

300 degC
T = 573.15 kelvin
573.15 kelvin


Next we'll create variables for $Q_n = 37,600 \frac{J}{mol}$ and the universal gas contant $R=8.31 \frac{J}{mol-K}$.

In [3]:
Qn = 37600 * u.J/u.mol
R = 8.31 * u.J/(u.mol*u.kelvin)

Our first nitrogen pressure is 0.10 MPa and our second nitrogen pressure is 5.0 MPa, we'll make variables for both

In [4]:
PN1 = 0.10
PN2 = 5.0

Now we can calculate the two nitrogren concentrations in wt% using the equation:

$$C_{N} = 4.9 \times 10^{-6} \sqrt{P_{N_2}} exp{\frac{Qn}{RT}} $$



In [5]:
CN1 = (4.9e-3)*sqrt(PN1)*exp(-Qn/(R*T))
CN1

5.777054779474043e-07

In [6]:
CN2 = (4.9e-3)*sqrt(PN2)*exp(-Qn/(R*T))
CN2

4.084994609852251e-06

These values ```CN1``` and ```CN2``` are in units of wt% N in an iron-nitrogen "alloy" where almost all of the alloy is iron with only a small amount of nitrogen. To use Fick's first law of diffusion:

$$ J = -D \frac{dC}{dx} $$

We need our concentrations C1 and C2 in units of mass per unit volume like kg/m<sup>3</sup> or g/cm<sup>3</sup> not in units of wt %.

To make the conversion between wt % and mass per unit volume we have to pick a mass of iron. This mass of iron will contain a mass of nitrogen. We can divide this mass of nitrogen by the volume of iron that corresponds to the mass of iron we picked. As long as we divide the mass of nitrogen by the volume of iron that contains that mass of nitrogen, we will end up with a unit conversion from wt% to kg/m<sup>3</sup> that works. So let's pick 1 kilogram of iron, and use the density of iron as 7.874 g/cm<sup>3</sup>

In [7]:
p=7.874*u.g/u.cm**3
p.ito(u.kg/u.m**3)
mFe = 1*u.kg
vFe = mFe/p

In [8]:
mN1 = mFe*CN1*0.01
CN1 = mN1/vFe
print(CN1)

mN2 = mFe*CN2*0.01
CN2 = mN2/vFe
print(CN2)

4.5488529333578606e-05 kilogram / meter ** 3
0.0003216524755797662 kilogram / meter ** 3


The difference in concentration dC, is just the difference between the two concentrations. dx the change in distance is the thickness of the plate, 1.5 mm.

In [9]:
dC = CN2-CN1
dx = 1.5 *u.mm
dx.ito(u.m)

Next we need to find the diffusion coefficient _D_. To do this we need the pre-exponential term _D<sub>0</sub>_ and the activating envery of diffusion _Q<sub>d</sub>_

From the beginning of the problem:

$D_0 = 5 \times 10^{-7} \frac{m^2}{s}$

$Q_d = 77,000 \frac{J}{mol} $

Let's assign these to variables with the applicable units.

In [10]:
D0 = 5e-7 * u.m**2/u.s
Qd = 77000 * u.J/u.mol

calculate diffusion constant _D_, according to:

$$ D = D_0e^{\frac{-Q_d}{RT}} $$

In [11]:
D = D0 * exp(-Qd/(R*T))

Calculate _J_ according to:

$$ J = -D \frac{dC}{dx} $$

In [12]:
J = -D*(dC/dx)
J

<Quantity(-8.768730355898267e-15, 'kilogram / meter ** 2 / second')>